In [5]:
import numpy as np
import abc

# constant
nodes_per_layer = [2,2,1]
bias_value = 1


def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_derivate(x):
    return x * (1 - x)

# Initialization techniques of weights here defined
class WeightInitialization(metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def init(self, lower, upper, shape):
        pass

class UniformDistributionWeight(WeightInitialization):

    def init(self, lower, upper, shape):
        return np.random.uniform(lower, upper, size=shape)


def get_nr_weights(nodes_per_layer):

    nr_weights = 0

    for idx in range(len(nodes_per_layer) - 1):
        current_layer = nodes_per_layer[idx]
        next_layer = nodes_per_layer[idx + 1]

        nr_weights = nr_weights + (current_layer + 1) * next_layer

    return nr_weights




In [56]:
class Bonita:

    def __init__(self, nodes_per_layer, activation_function, derivate_activation_function, learning_rate = 0.01, nr_epochs=10000):
        self.activation_function = activation_function
        self.derivate_activation_function = derivate_activation_function
        self.nodes_per_layer = nodes_per_layer
        self.lr = learning_rate
        self.y = []
        self.deltas = []
        self.epochs = nr_epochs

    def _divide_in_layers_matrix(self, weights, nodes_per_layer):


        layers = []
        last = 0
        for idx in range(len(nodes_per_layer) - 1):
            current_layer = nodes_per_layer[idx]
            next_layer = nodes_per_layer[idx + 1]
            temp = (current_layer + 1) * next_layer
            layers.append(weights[last : last + temp].reshape(current_layer + 1, next_layer))
            #print("{}:{}".format(last, last + temp))
            last = temp

        return layers




    def xor_net(self, x1, x2, weights):
        y = [[x1, x2, 1.0]]
        weights = self._divide_in_layers_matrix(weights, self.nodes_per_layer)

        for i in range(len(weights) - 1):
            activation = np.dot(y[i], weights[i])
            activity2 = self.activation_function(activation)
            print(activity2)
            activity2 = np.concatenate((np.array(activity2), np.array([1.0])))
            y.append(activity2)
        # porque é q esta merda ta fora do ciclo? bias?
        activation = np.dot(y[-1], weights[-1])
        activity2 = self.activation_function(activation)
        y.append(activity2)

        self.y.append(y)
        
            
        return y


    def mse(self, weights):

        self.inputs = np.array([[0.0, 0.0],[0.0,1.0],[1.0,0.0],[1.0,1.0]])
        self.expected_output = np.array([ [0.0],[1.0],[1.0],[0.0]])
        #w = self._divide_in_layers_matrix(weights, self.nodes_per_layer)

        val = 0
        for i in range(len(self.inputs)):
            predicted = self.xor_net(self.inputs[i][0], self.inputs[i][1], weights)
            val += (predicted[-1] - self.expected_output[i]) ** 2

        val = val/len(self.inputs)

        return val


    def grdmse(self, weights, incpt):

        index = 0

        error = self.expected_output[index] - incpt[-1]

        y = incpt
        delta_vec = [error * self.derivate_activation_function(y[-1])]

        w = self._divide_in_layers_matrix(weights, self.nodes_per_layer)

        for i in range(len(self.nodes_per_layer)-2, 0, -1):
            error = delta_vec[-1].dot(w[i][:-1].T)
            error = error*self.derivate_activation_function(y[i][:-1])
            delta_vec.append(error)

        # Now we need to set the values from back to front
        delta_vec.reverse()
        
        # Finally, we adjust the weights, using the backpropagation rules
        deltas = []
        for i in range(len(w)):
            layer = np.array(y[i]).reshape(1, self.nodes_per_layer[i]+1)
            delta = delta_vec[i].reshape(1, self.nodes_per_layer[i+1])
            deltas.append(layer.T.dot(delta))

            
        
        return deltas



    def gradient_descend(self, weights):
        #weights = self._divide_in_layers_matrix(weights, self.nodes_per_layer)

        for k in range(self.epochs):
            if (k+1) % 10000 == 0:
                print('epochs: {}'.format(k+1))
        

            self.mse(weights) # inputs and expected output completed in this function
            for possible_inputs in self.y:
                deltas = self.grdmse(weights, possible_inputs)
                print(np.array(deltas).flatten())
                #weights += self.lr * deltas.flatten()

            break
            


        



network = Bonita(nodes_per_layer, sigmoid, sigmoid_derivate, 0.01, nr_epochs=1000)
weights = UniformDistributionWeight().init(-1, 1, get_nr_weights(nodes_per_layer))
y = network.gradient_descend(weights)

       

[0.36957756 0.68977934]
[0.45217073 0.78879839]
[0.45236883 0.57201993]
[0.53768377 0.69183389]


/home/joao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:110: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


ValueError: could not broadcast input array from shape (3,2) into shape (3)

In [ ]:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def sigmoid_derivate(x):
    return x*(1-x)

layers = [2,2,1]
np.random.seed(0)

weights = []
for layer in range(len(layers) - 1):
    w = 2*np.random.rand(layers[layer] + 1, layers[layer+1]) - 1
    weights.append(w)


def forward_prop(x, activity, weights):
    y = x
    for i in range(len(weights)-1):
        activation = np.dot(y[i], weights[i])
        activity2 = activity(activation)
        print(activity2)
        # add the bias for the next layer
        activity2 = np.concatenate((np.ones(1), np.array(activity2)))
        y.append(activity2)

    activation = np.dot(y[-1], weights[-1])
    activity2 = activity(activation)
    y.append(activity2)
        
    return y


def back_prop( y, target, learning_rate, activity_derivative, weights):
    error = target - y[-1]
    delta_vec = [error * activity_derivative(y[-1])]
    print(y)

    # we need to begin from the back, from the next to last layer
    for i in range(len(layers)-2, 0, -1):
        error = delta_vec[-1].dot(weights[i][1:].T)
        error = error*activity_derivative(y[i][1:])
        delta_vec.append(error)

    # Now we need to set the values from back to front
    delta_vec.reverse()
    
    # Finally, we adjust the weights, using the backpropagation rules
    for i in range(len(weights)):
        layer = y[i].reshape(1, layers[i]+1)
        delta = delta_vec[i].reshape(1, layers[i+1])
        weights[i] += learning_rate*layer.T.dot(delta)

    return weights

def fit(data, labels, learning_rate=0.1, epochs=100, weights=[]):
    
    # Add bias units to the input layer - 
    # add a "1" to the input data (the always-on bias neuron)
    ones = np.ones((1, data.shape[0]))
    Z = np.concatenate((ones.T, data), axis=1)
    
    for k in range(epochs):
        if (k+1) % 10000 == 0:
            print('epochs: {}'.format(k+1))
    
        sample = np.random.randint(X.shape[0])

        # We will now go ahead and set up our feed-forward propagation:
        x = [Z[sample]]
        y = forward_prop(x, sigmoid, weights)

        # Now we do our back-propagation of the error to adjust the weights:
        target = labels[sample]
        weights = back_prop(y, target, learning_rate, sigmoid_derivate, weights)

X = np.array([[0, 0], [0, 1],
                [1, 0], [1, 1]])

# Set the labels, the correct results for the xor operation
Y = np.array([0, 1, 
                 1, 0])

# Call the fit function and train the network for a chosen number of epochs
fit(X, Y, epochs=50000, weights=weights)

[0.5752133  0.62718169]
[0.48651932 0.67336497]
[0.57605243 0.62806587]
[0.53924274 0.69412063]
[0.53711977 0.69228352]
[0.57381677 0.62596257]
[0.5241086 0.605799 ]
[0.48411794 0.6712209 ]
[0.48472882 0.67178403]
[0.48533439 0.67234158]
[0.48593469 0.67289363]
[0.52459783 0.60622784]
[0.48581114 0.67272261]
[0.52418038 0.60574908]
[0.53643537 0.69160472]
[0.52276594 0.60429026]
[0.53363964 0.6890902 ]
[0.48220729 0.66934538]
[0.57064268 0.62266146]
[0.53281621 0.68844342]
[0.53074785 0.68662237]
[0.5206184  0.60215236]
[0.56786185 0.61992904]
[0.52864694 0.68476034]
[0.51957541 0.60108839]
[0.47800326 0.66546711]
[0.52662346 0.68291665]
[0.47734721 0.66488228]
[0.52530532 0.68175683]
[0.52331209 0.67997146]
[0.5632435  0.61538472]
[0.5220147  0.67886701]
[0.51727617 0.59879786]
[0.47377136 0.66157402]
[0.47448551 0.66225176]
[0.47519409 0.66292341]
[0.52153123 0.67842509]
[0.51704242 0.59852876]
[0.51893437 0.67601909]
[0.55985591 0.61198358]
[0.51611567 0.59757758]
[0.51709851 0.6743

KeyboardInterrupt: 

In [ ]:
def predict_single_data(x, activity):
    val = np.concatenate((np.ones(1).T, np.array(x)))
    for i in range(0, len(weights)):
        val = activity(np.dot(val, weights[i]))
        val = np.concatenate((np.ones(1).T, np.array(val)))
    return val[1]


def predict(X):
    Y = np.array([]).reshape(0, layers[-1])
    for x in X:
        y = np.array([[predict_single_data(x)]])
        Y = np.vstack((Y,y))
    return Y

print("Final prediction")
for s in X:
    print(s, predict_single_data(s, sigmoid))


Final prediction
[0 0] 0.09861019261444225
[0 1] 0.9052667304918004
[1 0] 0.9024057587994762
[1 1] 0.11481992000443504
